PARTE 1: Abrir a página do pregão que contém as informações básicas e os links para cada item

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import requests
import datetime
import time
import pandas as pd
import csv
import os

In [74]:
#!/usr/bin/env python
# coding: utf-8


# URL da página do pregão
url = "https://contratos.sistema.gov.br/transparencia/compras?lei=LEI14133&lei_text=LEI14133&unidade_origem_id=9964&unidade_origem_id_text=160224+-+PARQUE+REGIONAL+DE+MANUTENCAO%2F5&modalidade_id=76&modalidade_id_text=05+-+Preg%C3%A3o"

# Obtendo a data atual
data_atual = datetime.date.today()

def iniciar():
    if os.path.exists('url_pregoes.csv'):
        try:
            url_pregoes_antiga = carregar_lista_pregoes()
                        
            url_pregoes_nova = baixar_nova_lista_pregoes(url)
            
            df_itens = verificar_se_ha_novos_pregoes(url_pregoes_antiga, url_pregoes_nova)
            
            salvar_dados(df_itens)
            
            return 'Dados Salvos com sucesso!'
        
        except Exception as e:
            print(f'Erro ao carregar lista de pregões: {e}.')
    else:
        print('Não há lista de pregões salvas nesse computador.')
        url_pregoes_antiga = []
        
        #Baixando lista de pregões
        print('Obtendo a url dos pregões...')
        url_pregoes_nova = baixar_nova_lista_pregoes(url)
        
        print('Obtendo a url dos itens...')      
        url_itens = obter_url_dos_itens(url_pregoes_nova)
        print('URL dos itens obtida com sucesso!') 
        
        print('Obtendo os dados dos itens...')
        df_itens = atualizar_todos_os_dados(url_itens)
        print('Dados dos itens obtidos com sucesso!')
    
        salvar_dados(df_itens)
        
        return 'Dados Salvos com sucesso!'

In [8]:
def carregar_lista_pregoes():
    nome_arquivo = 'url_pregoes.csv'
    url_pregoes_antiga = []
    
    with open(nome_arquivo, mode='r') as file:
        reader = csv.reader(file)
        next(reader)  # Pular o cabeçalho
        url_pregoes_antiga = [row[0] for row in reader]
    
    print('Lista de Pregões salvas no PC encontrada!')
        
    return url_pregoes_antiga

def baixar_nova_lista_pregoes(url):
    
    print('Baixando nova lista de url dos pregões ...')
    # Configuração do WebDriver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Executar em modo headless para não abrir uma janela do navegador
    driver = webdriver.Chrome(options=options)
    driver.get(url)

    # Selecionar a opção "Todos"
    select = Select(driver.find_element(By.NAME, 'crudTable_length'))
    select.select_by_value('-1')
    time.sleep(5)  # Esperar alguns segundos para garantir que a página carregue completamente

    # Obter e parsear o HTML da página
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()

    specific_url_part = 'https://contratos.sistema.gov.br/transparencia/compras/'
    url_pregoes_nova = [a_tag['href'].replace('show', 'itens') for a_tag in soup.find_all('a', href=True) if specific_url_part in a_tag['href']]
    


    # Gravar a lista em um arquivo CSV
    nome_arquivo = 'url_pregoes.csv'
    with open(nome_arquivo, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([f'Atualizado em {data_atual}'])  # Escrever a data da atualização
        writer.writerows([[url] for url in url_pregoes_nova])
        print('Nova Lista de url dos pregões salva com sucesso!')
    
    return url_pregoes_nova

In [34]:
def verificar_se_ha_novos_pregoes(url_pregoes_antiga, url_pregoes_nova):
    
    novos_pregoes = list(set(url_pregoes_nova) - set(url_pregoes_antiga))
    
    if novos_pregoes == []:
        
        print('Não há novos pregões. Iniciando atualização dos saldos...')
        
            #Carregando df_itens
    
        print('Carregando DataFrame com os dados dos itens...')

        df_itens = pd.read_csv('df_itens.csv', sep=';')
        #Para obter as informações de cada item

        #chamar função de atualizar o saldo dos pregões existentes
    
        df_itens = atualizar_saldo(df_itens)
        
        return df_itens
        
        
    else:
        
        print('Há novos pregões não cadastrados! \n Obtendo os dados dos novos pregões...') 
        
        url_itens_novos = obter_url_dos_itens(novos_pregoes)

        df_itens_novos = atualizar_todos_os_dados(url_itens_novos)
        
        print('Dados dos novos pregões atualizados! Atualizando o saldo dos pregões anteriores...')
        
        
        #colocar um try encontrar df_itens salvos
        df_itens_antigos = pd.read_csv('df_itens.csv', sep = ';')
        
        df_itens_antigos = atualizar_saldo(df_itens_antigos)
        
        print('Saldo dos pregões anteriores atualizado. Concatenando a tabela dos itens antigos com os novos...')
        
        df_itens = pd.concat([df_itens_novos, df_itens_antigos])
        
        return df_itens

In [5]:
def salvar_dados(df_itens):
    
    #Salvando o df em arquivo csv
    nome_arquivo = "df_itens.csv"

    # Exportando o DataFrame para um arquivo CSV com delimitador ';' e codificação UTF-8
    df_itens.to_csv(nome_arquivo, sep=';', encoding='utf-8', index=False)

    print('Arquivo com a tabela dos itens salva com sucesso!')
    
    #Gravar a lista em um arquivo CSV
    
    df_itens['Link do Item'].to_csv('url_itens.csv', sep=';', encoding='utf-8', index=False)
    
    print('Lista com a url dos itens salva com sucesso!')

In [6]:
def obter_url_dos_itens(novos_pregoes):
    
    url_itens = []

    for url in novos_pregoes:       

    # Configuração do WebDriver
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')  # Executar em modo headless para não abrir uma janela do navegador
        driver = webdriver.Chrome(options=options)
        driver.get(url)

        # Encontrar o elemento select pelo ID
        select_element = driver.find_element(By.NAME, 'crudTable_length')

        # Criar um objeto Select
        select = Select(select_element)

        # Selecionar a opção "Todos"
        select.select_by_value('-1')

        # Esperar alguns segundos para garantir que a página carregue completamente
        time.sleep(5)

        # Obter o HTML da página
        page_source = driver.page_source

        # Fechar o WebDriver
        driver.quit()

        # Parsear o HTML com Beautiful Soup
        soup = BeautifulSoup(page_source, 'html.parser')

        # Encontrar os links para cada item
        specific_url_part = 'https://contratos.sistema.gov.br/transparencia/compras/'

        for a_tag in soup.find_all('a', href=True):
            href = a_tag['href']
            if specific_url_part in href:
                url_itens.append(href)    

    url_itens = [url for url in url_itens if 'show' in url]
    
        
    return url_itens

In [16]:
def atualizar_todos_os_dados(url_itens):

    linhas = []

    for url in url_itens:
        
        # Enviar uma solicitação GET para obter o conteúdo da página
        response = requests.get(url)
        
        # Verificar se a solicitação foi bem-sucedida

        if response.status_code == 200:              

            # Parsear o HTML da página com Beautiful Soup
            soup = BeautifulSoup(response.content, 'html.parser')

            #Pregão
            pregao = soup.find_all('div', class_='header-title')[1].text.strip().replace('Itens da compra: 160224 - ','')

            # Encontrar todas as tabelas na página
            tables = soup.find_all('table')
            #Define a nova linha
            linha = [
                pregao, #Pregão
                tables[0].find_all('span')[0].text.strip(), #Item
                tables[0].find_all('span')[2].text.strip(), #Descrição
                tables[0].find_all('span')[3].text.strip(), #Descrição detalhada
                #tables[0].find_all('span')[4].text.strip(), #Qtd. Total
                tables[0].find_all('span')[5].text.strip(), #Vig. Início ARP
                tables[0].find_all('span')[6].text.strip(), #Vig. Fim ARP
                tables[1].find_all('td')[0].text.strip(), #Unidade
                #tables[1].find_all('td')[1].text.strip(), #Tipo UASG
                tables[1].find_all('td')[2].text.strip(), #Qtd. Autorizada

                tables[2].find_all('td')[0].text.strip(), #Fornecedor
                #tables[2].find_all('td')[1].text.strip(), #Qtd. Homologada
                tables[2].find_all('td')[2].text.strip(), #Val. Unitário
                #tables[2].find_all('td')[3].text.strip(), #Val. Negociado      
                tables[1].find_all('td')[3].text.strip(), #Qtd. Saldo
                url, #link de cada item no Comprasgov
                ]
            #Append a linha
            linhas.append(linha)
            
    #Para obter os cabeçalhos

    headers = [
        "Número da Compra",
        tables[0].find_all('strong')[0].text.strip(), #Item
        tables[0].find_all('strong')[2].text.strip(), #Descrição
        tables[0].find_all('strong')[3].text.strip(), #Descrição detalhada
        #tables[0].find_all('strong')[4].text.strip(), #Qtd. Total
        tables[0].find_all('strong')[5].text.strip(), #Vig. Início ARP
        tables[0].find_all('strong')[6].text.strip(), #Vig. Fim ARP
        tables[1].find_all('th')[0].text.strip(), #Unidade
        #tables[1].find_all('th')[1].text.strip(), #Tipo UASG
        tables[1].find_all('th')[2].text.strip(), #Qtd. Autorizada

        tables[2].find_all('th')[0].text.strip(), #Fornecedor
        #tables[2].find_all('th')[1].text.strip(), #Qtd. Homologada
        tables[2].find_all('th')[2].text.strip(), #Val. Unitário
        #tables[2].find_all('th')[3].text.strip(), #Val. Negociado
        tables[1].find_all('th')[3].text.strip(), #Qtd. Saldo
        'Link do Item'
    ]

    headers[1] = "Número do Item"
    headers[2] = "Descrição"
    headers[3] = "Descrição Detalhada"
    headers[4] = "Início da Vigência"
    headers[5] = "Fim da Vigência"

    #transformando os dados em um dataframe

    df_itens = pd.DataFrame(linhas, columns = headers)

    #Corrigindo os valores que são numéricos

    df_itens['Qtd. Autorizada'] = df_itens['Qtd. Autorizada'].str.replace('.','')
    df_itens['Qtd. Autorizada'] = df_itens['Qtd. Autorizada'].str.replace(',','.')

    df_itens['Val. Unitário'] = df_itens['Val. Unitário'].str.replace('.','')
    df_itens['Val. Unitário'] = df_itens['Val. Unitário'].str.replace(',','.')

    df_itens['Qtd. Saldo'] = df_itens['Qtd. Saldo'].str.replace('.','')
    df_itens['Qtd. Saldo'] = df_itens['Qtd. Saldo'].str.replace(',','.')

    #Inserindo novas colunas

    tipo_compra = []
    numero_compra_pregao = []
    ano_compra_pregao = []


    for i in range(len(df_itens['Número da Compra'])):
        tipo = df_itens['Número da Compra'][i].split(' ')[0]
        numero_pregao = df_itens['Número da Compra'][i].split(' ')[2].split('/')[0]
        ano_pregao = df_itens['Número da Compra'][i].split(' ')[2].split('/')[1]

        tipo_compra.append(tipo)
        numero_compra_pregao.append(numero_pregao)
        ano_compra_pregao.append(ano_pregao)

    df_itens['Tipo de Compra'] = tipo_compra
    df_itens['Número do Pregão'] = numero_compra_pregao
    df_itens['Ano do Pregão'] = ano_compra_pregao

    return df_itens

In [69]:
def atualizar_saldo(df_itens):
    
#atualizando o saldo
    
    for i in range(len(df_itens['Link do Item'])):
        
        url = df_itens['Link do Item'][i]
        
        # Enviar uma solicitação GET para obter o conteúdo da página
        response = requests.get(url)
        
        # Verificar se a solicitação foi bem-sucedida
    
        if response.status_code == 200:  

            # Parsear o HTML da página com Beautiful Soup
            soup = BeautifulSoup(response.content, 'html.parser')

            # Encontrar todas as tabelas na página
            tables = soup.find_all('table')

            #Define a nova linha
            saldo = tables[1].find_all('td')[3].text.strip() #Qtd. Saldo

            #Gravar saldo
            df_itens['Qtd. Saldo'][i] = saldo

            print('Saldo do item ' + str(i) + ' atualizado com sucesso!')

    df_itens['Qtd. Saldo'] = df_itens['Qtd. Saldo'].str.replace('.','')
    df_itens['Qtd. Saldo'] = df_itens['Qtd. Saldo'].str.replace(',','.')
    
    return df_itens

In [ ]:
if __name__ == "__main__":
    iniciar()